# 1 The Half-edge Data Structure

In [1]:
from halfedge_mesh.halfedge_mesh import * #import a very lightweight package for half-edge data structures
%load_ext autoreload
%autoreload 2

### 1 Generate mesh for our cube

In [2]:
def create_halfedge_cube():
    ################### 1. Initialise ################################
    cube = HalfedgeMesh()
    cube.update_vertices([
        [-1, -1, -1], [-1, -1, 1], [-1, 1, 1], [-1, 1, -1],
        [1, -1, -1], [1, -1, 1], [1, 1, 1], [1, 1, -1]
    ])
    cube.facets = [Facet(index=i) for i in range(6)]
    cube.halfedges = [Halfedge(index=i) for i in range(24)]
    
    ################## 2. Define Halfedge Connectivity ################  
    # Back face
    cube.halfedges[0].update(vertex=cube.vertices[0], next=cube.halfedges[1], facet=cube.facets[0], opposite=cube.halfedges[10])
    cube.halfedges[1].update(vertex=cube.vertices[3], next=cube.halfedges[2], facet=cube.facets[0], opposite=cube.halfedges[23])
    cube.halfedges[2].update(vertex=cube.vertices[7], next=cube.halfedges[3], facet=cube.facets[0], opposite=cube.halfedges[12])
    cube.halfedges[3].update(vertex=cube.vertices[4], next=cube.halfedges[0], facet=cube.facets[0], opposite=cube.halfedges[17]) 
    # Front face
    cube.halfedges[4].update(vertex=cube.vertices[2], next=cube.halfedges[5], facet=cube.facets[1], opposite=cube.halfedges[14])
    cube.halfedges[5].update(vertex=cube.vertices[1], next=cube.halfedges[6], facet=cube.facets[1], opposite=cube.halfedges[19])
    cube.halfedges[6].update(vertex=cube.vertices[5], next=cube.halfedges[7], facet=cube.facets[1], opposite=cube.halfedges[8])
    cube.halfedges[7].update(vertex=cube.vertices[6], next=cube.halfedges[4], facet=cube.facets[1], opposite=cube.halfedges[21])
    # Bottom face
    cube.halfedges[8].update(vertex=cube.vertices[1], next=cube.halfedges[9], facet=cube.facets[2], opposite=cube.halfedges[6])
    cube.halfedges[9].update(vertex=cube.vertices[0], next=cube.halfedges[10], facet=cube.facets[2], opposite=cube.halfedges[20])
    cube.halfedges[10].update(vertex=cube.vertices[4], next=cube.halfedges[11], facet=cube.facets[2], opposite=cube.halfedges[0])
    cube.halfedges[11].update(vertex=cube.vertices[5], next=cube.halfedges[8], facet=cube.facets[2], opposite=cube.halfedges[16])
    # Top face
    cube.halfedges[12].update(vertex=cube.vertices[3], next=cube.halfedges[13], facet=cube.facets[3], opposite=cube.halfedges[2])
    cube.halfedges[13].update(vertex=cube.vertices[2], next=cube.halfedges[14], facet=cube.facets[3], opposite=cube.halfedges[22])
    cube.halfedges[14].update(vertex=cube.vertices[6], next=cube.halfedges[15], facet=cube.facets[3], opposite=cube.halfedges[4])
    cube.halfedges[15].update(vertex=cube.vertices[7], next=cube.halfedges[12], facet=cube.facets[3], opposite=cube.halfedges[18])
    # Right face
    cube.halfedges[16].update(vertex=cube.vertices[4], next=cube.halfedges[17], facet=cube.facets[4], opposite=cube.halfedges[11])
    cube.halfedges[17].update(vertex=cube.vertices[7], next=cube.halfedges[18], facet=cube.facets[4], opposite=cube.halfedges[3])
    cube.halfedges[18].update(vertex=cube.vertices[6], next=cube.halfedges[19], facet=cube.facets[4], opposite=cube.halfedges[15])
    cube.halfedges[19].update(vertex=cube.vertices[5], next=cube.halfedges[16], facet=cube.facets[4], opposite=cube.halfedges[5])
    # Left face
    cube.halfedges[20].update(vertex=cube.vertices[1], next=cube.halfedges[21], facet=cube.facets[5], opposite=cube.halfedges[9])
    cube.halfedges[21].update(vertex=cube.vertices[2], next=cube.halfedges[22], facet=cube.facets[5], opposite=cube.halfedges[7])
    cube.halfedges[22].update(vertex=cube.vertices[3], next=cube.halfedges[23], facet=cube.facets[5], opposite=cube.halfedges[13])
    cube.halfedges[23].update(vertex=cube.vertices[0], next=cube.halfedges[20], facet=cube.facets[5], opposite=cube.halfedges[1])

    ########## 3. Define Facet Connectivity ##############
    cube.facets[0].update(halfedge=cube.halfedges[0])
    cube.facets[1].update(halfedge=cube.halfedges[4])
    cube.facets[2].update(halfedge=cube.halfedges[8])
    cube.facets[3].update(halfedge=cube.halfedges[12])
    cube.facets[4].update(halfedge=cube.halfedges[16])
    cube.facets[5].update(halfedge=cube.halfedges[20])
    
    ########## 4. Define Vertex Connectivity ##############
    cube.vertices[0].update(halfedge=cube.halfedges[0])
    cube.vertices[1].update(halfedge=cube.halfedges[7])
    cube.vertices[2].update(halfedge=cube.halfedges[13])
    cube.vertices[3].update(halfedge=cube.halfedges[12])
    cube.vertices[4].update(halfedge=cube.halfedges[3])
    cube.vertices[5].update(halfedge=cube.halfedges[19])
    cube.vertices[6].update(halfedge=cube.halfedges[5])
    cube.vertices[7].update(halfedge=cube.halfedges[17])
    
    return cube

cube = create_halfedge_cube()
cube.write_off('my_cube.off') #This converts the halfedge triangle into a .off file. Open it in Meshlab.


### 2

In [3]:
def calculate_centroid(face):
    """计算面的质心"""
    vertices = face.get_vertices()
    x = sum(vertex.x for vertex in vertices) / len(vertices)
    y = sum(vertex.y for vertex in vertices) / len(vertices)
    z = sum(vertex.z for vertex in vertices) / len(vertices)
    return [x, y, z]

## 解释一下facet.get_vertices()
## 就是先找到面对应的那个halfedge，然后通过不断调用next halfedge找到面所有的halfedge的起点，也就找到了面的所有vertices



### 3

In [4]:
def create_halfedge_tetrahedron():
    ################### 1. Initialise ################################
    
    tetra = HalfedgeMesh() #Create a halfedge-mesh object
    tetra.update_vertices([ [1,0,-2**(-0.5)], [-1,0,-2**(-0.5)], [0,1,2**(-0.5)] , [0,-1,2**(-0.5)]]) #Create vertices
    tetra.facets = [ Facet(index=i) for i in range(4) ] 
    tetra.halfedges = [ Halfedge(index = i) for i in range(12) ] 
    
    ################## 2. Define Halfedge Connectivity ################
    
    tetra.halfedges[0].update ( vertex = tetra.vertices[0], next = tetra.halfedges[1] , facet = tetra.facets[0] ,
                               opposite = tetra.halfedges[3] )
    
    tetra.halfedges[1].update ( vertex =  tetra.vertices[1], next = tetra.halfedges[2], facet = tetra.facets[0] ,
                               opposite = tetra.halfedges[5])
    
    tetra.halfedges[2].update ( vertex =  tetra.vertices[2], next = tetra.halfedges[0], facet = tetra.facets[0] ,
                              opposite = tetra.halfedges[4]) 
    
    tetra.halfedges[3].update ( vertex = tetra.vertices[1], next = tetra.halfedges[7], facet = tetra.facets[3] ,
                              opposite = tetra.halfedges[0])
    
    tetra.halfedges[4].update ( vertex = tetra.vertices[0], next = tetra.halfedges[6], facet = tetra.facets[1], 
                               opposite = tetra.halfedges[2] ) 
    
    tetra.halfedges[5].update ( vertex = tetra.vertices[2], next = tetra.halfedges[9], facet = tetra.facets[2],
                               opposite =  tetra.halfedges[1])
    
    tetra.halfedges[6].update ( vertex = tetra.vertices[2], next = tetra.halfedges[11] , facet = tetra.facets[1],
                               opposite =  tetra.halfedges[10])
    
    tetra.halfedges[7].update ( vertex = tetra.vertices[0], next = tetra.halfedges[8], facet = tetra.facets[3],
                               opposite =  tetra.halfedges[11])
    
    tetra.halfedges[8].update ( vertex = tetra.vertices[3], next = tetra.halfedges[3], facet = tetra.facets[3],
                               opposite = tetra.halfedges[9] )

    tetra.halfedges[9].update ( vertex = tetra.vertices[1], next = tetra.halfedges[10], facet = tetra.facets[2],
                               opposite = tetra.halfedges[8] )
    
    tetra.halfedges[10].update ( vertex = tetra.vertices[3], next = tetra.halfedges[5], facet = tetra.facets[2],
                                opposite =  tetra.halfedges[6])
    
    tetra.halfedges[11].update ( vertex = tetra.vertices[3], next = tetra.halfedges[4], facet = tetra.facets[1],
                                opposite = tetra.halfedges[7] )
    
    
    ########## 3. Define Facet Connectivity ##############
    
    tetra.facets[0].update (halfedge = tetra.halfedges[0])
    tetra.facets[1].update (halfedge = tetra.halfedges[6])
    tetra.facets[2].update (halfedge = tetra.halfedges[10])
    tetra.facets[3].update (halfedge = tetra.halfedges[7])

    ########## 4. Define Vertex Connectivity ##############

    tetra.vertices[0].update (halfedge = tetra.halfedges[0])
    tetra.vertices[1].update (halfedge = tetra.halfedges[1])
    tetra.vertices[2].update (halfedge = tetra.halfedges[2])
    tetra.vertices[3].update (halfedge = tetra.halfedges[8])

    return tetra

tetra = create_halfedge_tetrahedron()
tetra.write_off('my_tetrahedron.off') 

In [5]:
import numpy as np
import math

def sort_halfedges_ccw(halfedges, vertex, mesh):
    # 计算顶点的平均法线向量
    normal = np.array([0.0, 0.0, 0.0])
    valid_facets = 0
    for he in halfedges:
        if he.facet:
            facet_normal = he.facet.get_normal()
            normal += np.array(facet_normal)
            valid_facets += 1
    if valid_facets > 0:
        normal /= valid_facets

    # 选择一个与平均法线向量垂直的参考向量
    ref_vector = np.array([0.0, 0.0, 1.0]) if abs(normal[2]) < 1.0 else np.array([0.0, 1.0, 0.0])

    # 计算参考向量在平均法线方向的投影，然后从参考向量中减去该投影
    proj_length = np.dot(ref_vector, normal)
    proj = proj_length * normal
    ref_dir = normalize(ref_vector - proj)

    # 计算每个半边的中心点，并根据它们相对于参考向量的角度进行排序
    angles = []
    for he in halfedges:
        if he.facet:
            facet_center = calculate_centroid(he.facet)
            center_vector = np.array(facet_center) - np.array([vertex.x, vertex.y, vertex.z])
            center_vector -= np.dot(center_vector, normal) * normal  # 保证向量在平面内
            center_vector = normalize(center_vector)
            angle = math.atan2(np.linalg.norm(np.cross(ref_dir, center_vector)), np.dot(ref_dir, center_vector))
            angles.append((angle, he))

    # 按角度排序
    sorted_halfedges = [he for _, he in sorted(angles, key=lambda x: x[0])]
    return sorted_halfedges

def compute_dual_mesh(original_mesh):
    dual_vertices = []  # 对偶图的顶点列表
    dual_halfedges = []  # 对偶图的半边列表
    dual_facets = []  # 对偶图的面列表

    # 为原始网格的每个面创建一个对偶顶点
    for facet in original_mesh.facets:
        center = calculate_centroid(facet)  # 计算质心
        dual_vertex = Vertex(center[0], center[1], center[2], len(dual_vertices))
        dual_vertices.append(dual_vertex)

    # 为原始网格的每个顶点创建一个对偶面，并创建对偶半边
    for vertex in original_mesh.vertices:
        new_facet = Facet(index=vertex.index)  # 创建对偶面
        dual_facets.append(new_facet)

        # 找到以该顶点为起点的所有半边
        start_halfedges = [he for he in original_mesh.halfedges if he.vertex == vertex]

        # 按逆时针顺序排序半边
        sorted_halfedges = sort_halfedges_ccw(start_halfedges, vertex, original_mesh)

        first_dual_he = None
        prev_dual_he = None

        # 为每个半边创建对偶半边，并设置其next和prev关系
        for i, he in enumerate(sorted_halfedges):
            if he.facet and he.opposite and he.opposite.facet:
                dual_start_vertex = dual_vertices[he.facet.index]
                dual_end_vertex = dual_vertices[he.opposite.facet.index]

                dual_he = Halfedge(vertex=dual_start_vertex, facet=new_facet)
                dual_halfedges.append(dual_he)

                if not dual_start_vertex.halfedge:
                    dual_start_vertex.halfedge = dual_he

                if i == 0:
                    new_facet.halfedge = dual_he
                    first_dual_he = dual_he
                else:
                    prev_dual_he.next = dual_he
                    dual_he.prev = prev_dual_he

                prev_dual_he = dual_he

        if first_dual_he and prev_dual_he:
            prev_dual_he.next = first_dual_he
            first_dual_he.prev = prev_dual_he

    for he in dual_halfedges:
        for candidate in dual_halfedges:
            if he.vertex == candidate.next.vertex and he.next.vertex == candidate.vertex:
                he.opposite = candidate
                candidate.opposite = he
                break

    dual_mesh = HalfedgeMesh(vertices=dual_vertices, halfedges=dual_halfedges, facets=dual_facets)
    return dual_mesh


dual_cube = compute_dual_mesh(cube)
print("asd")
dual_cube.write_off('my_dual_cubeeeee.off') #This converts the halfedge triangle into a .off file. Open it in Meshlab.


AttributeError: 'NoneType' object has no attribute 'vertex'

In [32]:
def calculate_original_mesh_centroid(original_mesh):
    # 初始化累加器
    sum_x = 0
    sum_y = 0
    sum_z = 0

    # 遍历所有顶点，累加它们的坐标
    for vertex in original_mesh.vertices:
        sum_x += vertex.x
        sum_y += vertex.y
        sum_z += vertex.z

    # 计算顶点的总数
    num_vertices = len(original_mesh.vertices)

    # 如果顶点总数大于0，则计算平均坐标
    if num_vertices > 0:
        centroid_x = sum_x / num_vertices
        centroid_y = sum_y / num_vertices
        centroid_z = sum_z / num_vertices
        return np.array([centroid_x, centroid_y, centroid_z])
    else:
        # 如果网格中没有顶点，则返回原点作为质心
        return np.array([0, 0, 0])


def correct_halfedges_order(dual_facet, original_mesh_centroid):
    # 计算对偶面的质心
    dual_facet_centroid = calculate_centroid(dual_facet)

    # 获取对偶面的所有半边
    halfedges = []
    start_he = dual_facet.halfedge
    he = start_he
    while True:
        halfedges.append(he)
        he = he.next
        if he == start_he:
            break

    # 计算从对偶面质心到原网格质心的向量
    to_original_centroid_vector = original_mesh_centroid - dual_facet_centroid

    # 计算第一个半边的法线和向量的点乘
    first_he_normal = dual_facet.get_normal()
    dot_product = np.dot(first_he_normal, to_original_centroid_vector)

    # 如果点乘小于0，反转半边的顺序
    if dot_product > 0:
        halfedges.reverse()

        # 重新设置每个半边的next和prev属性
        for i, he in enumerate(halfedges):
            next_index = (i + 1) % len(halfedges)
            prev_index = (i - 1 + len(halfedges)) % len(halfedges)
            he.next = halfedges[next_index]
            he.prev = halfedges[prev_index]

        # 更新对偶面的halfedge为反转后的第一个半边
        dual_facet.halfedge = halfedges[0]

# 注意: calculate_facet_centroid, calculate_halfedge_normal 需要您自己实现
# calculate_facet_centroid(dual_facet) 应计算并返回对偶面的质心
# calculate_halfedge_normal(halfedge) 应计算并返回半边的法线




# 比上面的加上了vertex的半边属性，但是我发现有的面是反的（黑的）
def compute_dual_mesh(original_mesh):
    dual_vertices = []  # 对偶图的顶点列表
    dual_halfedges = []  # 对偶图的半边列表
    dual_facets = []  # 对偶图的面列表

    # 为原始网格的每个面创建一个对偶顶点
    for facet in original_mesh.facets:
        center = calculate_centroid(facet)  # 计算质心
        dual_vertex = Vertex(center[0], center[1], center[2], len(dual_vertices))
        dual_vertices.append(dual_vertex)

    # 为原始网格的每个顶点创建一个对偶面，并创建对偶半边
    for vertex in original_mesh.vertices:
        new_facet = Facet(index=vertex.index)  # 创建对偶面
        dual_facets.append(new_facet)

        # 找到以该顶点为起点的所有半边
        start_halfedges = [he for he in original_mesh.halfedges if he.vertex == vertex]
        first_dual_he = None  # 用于闭合循环，连接最后一个和第一个对偶半边

        # 为每个半边创建对偶半边，并设置其next和prev关系
        for i, he in enumerate(start_halfedges):
            if he.facet and he.opposite and he.opposite.facet:
                dual_start_vertex = dual_vertices[he.facet.index]
                dual_end_vertex = dual_vertices[he.opposite.facet.index]

                dual_he = Halfedge(vertex=dual_start_vertex, facet=new_facet)
                dual_halfedges.append(dual_he)

                # 为对偶顶点设置halfedge属性
                if not dual_start_vertex.halfedge:
                    dual_start_vertex.halfedge = dual_he

                if i == 0:
                    new_facet.halfedge = dual_he  # 设置对偶面的第一个半边
                    first_dual_he = dual_he
                else:
                    prev_dual_he.next = dual_he  # 设置上一个对偶半边的next属性
                    dual_he.prev = prev_dual_he  # 设置当前对偶半边的prev属性

                prev_dual_he = dual_he  # 更新上一个对偶半边为当前对偶半边

        if first_dual_he and prev_dual_he:
            # 闭合循环：连接最后一个和第一个对偶半边
            prev_dual_he.next = first_dual_he
            first_dual_he.prev = prev_dual_he
            
        # 修正为逆时针方向
        correct_halfedges_order(new_facet, calculate_original_mesh_centroid(original_mesh))


    # 为每个对偶半边设置opposite属性
    for he in dual_halfedges:
        for candidate in dual_halfedges:
            if he.vertex == candidate.next.vertex and he.next.vertex == candidate.vertex:
                he.opposite = candidate
                candidate.opposite = he
                break

    dual_mesh = HalfedgeMesh(vertices=dual_vertices, halfedges=dual_halfedges, facets=dual_facets)
    
    return dual_mesh




dual_cube = compute_dual_mesh(cube)
dual_cube1 = compute_dual_mesh(dual_cube)
print(len(dual_cube1.facets))
print("asd")
dual_cube1.write_off('my_dual_cub.off') #This converts the halfedge triangle into a .off file. Open it in Meshlab.


6
asd


## Task 1: Construct a halfedge mesh for a tetrahedron.

 - First, draw a diagram on paper to plan out how you will define the vertices, halfedges and facets.
 - Then fill in the following function to generate a halfedge mesh for a tetrahedron.

Hint: If you are confused about how a halfedge mesh works, have a look at this interactive webpage: https://jerryyin.info/geometry-processing-algorithms/half-edge/

- To understand why the connections are defined like this, you will need to look at the image 'tetrahedron'

In [ ]:
def create_halfedge_tetrahedron():
    ################### 1. Initialise ################################
    
    tetra = HalfedgeMesh() #Create a halfedge-mesh object
    tetra.update_vertices([ [1,0,-2**(-0.5)], [-1,0,-2**(-0.5)], [0,1,2**(-0.5)] , [0,-1,2**(-0.5)]]) #Create vertices
    tetra.facets = [ Facet(index=i) for i in range(4) ] 
    tetra.halfedges = [ Halfedge(index = i) for i in range(12) ] 
    
    ################## 2. Define Halfedge Connectivity ################
    
    tetra.halfedges[0].update ( vertex = tetra.vertices[0], next = tetra.halfedges[1] , facet = tetra.facets[0] ,
                               opposite = tetra.halfedges[3] )
    
    tetra.halfedges[1].update ( vertex =  tetra.vertices[1], next = tetra.halfedges[2], facet = tetra.facets[0] ,
                               opposite = tetra.halfedges[5])
    
    tetra.halfedges[2].update ( vertex =  tetra.vertices[2], next = tetra.halfedges[0], facet = tetra.facets[0] ,
                              opposite = tetra.halfedges[4]) 
    
    tetra.halfedges[3].update ( vertex = tetra.vertices[1], next = tetra.halfedges[7], facet = tetra.facets[3] ,
                              opposite = tetra.halfedges[0])
    
    tetra.halfedges[4].update ( vertex = tetra.vertices[0], next = tetra.halfedges[6], facet = tetra.facets[1], 
                               opposite = tetra.halfedges[2] ) 
    
    tetra.halfedges[5].update ( vertex = tetra.vertices[2], next = tetra.halfedges[9], facet = tetra.facets[2],
                               opposite =  tetra.halfedges[1])
    
    tetra.halfedges[6].update ( vertex = tetra.vertices[2], next = tetra.halfedges[11] , facet = tetra.facets[1],
                               opposite =  tetra.halfedges[10])
    
    tetra.halfedges[7].update ( vertex = tetra.vertices[0], next = tetra.halfedges[8], facet = tetra.facets[3],
                               opposite =  tetra.halfedges[11])
    
    tetra.halfedges[8].update ( vertex = tetra.vertices[3], next = tetra.halfedges[3], facet = tetra.facets[3],
                               opposite = tetra.halfedges[9] )

    tetra.halfedges[9].update ( vertex = tetra.vertices[1], next = tetra.halfedges[10], facet = tetra.facets[2],
                               opposite = tetra.halfedges[8] )
    
    tetra.halfedges[10].update ( vertex = tetra.vertices[3], next = tetra.halfedges[5], facet = tetra.facets[2],
                                opposite =  tetra.halfedges[6])
    
    tetra.halfedges[11].update ( vertex = tetra.vertices[3], next = tetra.halfedges[4], facet = tetra.facets[1],
                                opposite = tetra.halfedges[7] )
    
    
    ########## 3. Define Facet Connectivity ##############
    
    tetra.facets[0].update (halfedge = tetra.halfedges[0])
    tetra.facets[1].update (halfedge = tetra.halfedges[6])
    tetra.facets[2].update (halfedge = tetra.halfedges[10])
    tetra.facets[3].update (halfedge = tetra.halfedges[7])

    ########## 4. Define Vertex Connectivity ##############

    tetra.vertices[0].update (halfedge = tetra.halfedges[0])
    tetra.vertices[1].update (halfedge = tetra.halfedges[1])
    tetra.vertices[2].update (halfedge = tetra.halfedges[2])
    tetra.vertices[3].update (halfedge = tetra.halfedges[8])

    return tetra

 - Test your function below. Open the mesh file in Meshlab to check it. Does it look like a tetrahedron?

In [ ]:
tetra = create_halfedge_tetrahedron()
tetra.write_off('my_tetrahedron.off') 

Note: If the mesh appears dark, it is 'inside out' - you can fix this by changing the order of the vertices, or you can use the method HalfedgeMesh.flip(), which multiplies the x-co-ordinate of every vertex by minus 1.

## Task 2: find the vertices in the one-ring of a vertex

 - Given a halfedge mesh and a starting vertex index, this function should return the pointers to all the vertices in the onering, in order.
This can be done very efficiently by exploiting the halfedge connectivity.

Hint: It can be useful to print out indices of halfedges and vertices using e.g. print(current_halfedge.index).

Hint: You can make chains of pointers to move around the graph, e.g. current_halfedge.opposite.opposite.next.vertex.halfedge.facet

As always, drawing a diagram is useful.

In [ ]:
def find_onering(origin_vertex):
    
    origin_halfedge = origin_vertex.halfedge
    
    onering_vertices = []
    halfedge_list = []
    
    cur_halfedge = origin_halfedge
    while True:
        halfedge_list.append(cur_halfedge.index)#helpful for debugging
        print()
        onering_vertices.append(cur_halfedge.opposite.vertex)
        cur_halfedge = cur_halfedge.opposite.next
        if cur_halfedge == origin_halfedge:
            break
            
    #print('Halfedge traversal: ',halfedge_list)
        
    return onering_vertices
        

 - Test your code on the tetraheron and on the icosphere (download the icosphere mesh 'sphere4.off' from the github page).

 - For the tetrahedron, check your diagram. Are the onering vertices correct?

 - For the icosphere, is the number of onering vertices as you expect? (You will need to look at the mesh in Meshlab.)

In [ ]:
i=0
onering = find_onering(tetra.vertices[i])#Are the onering vertices correct? Check all four vertices.
print ('onering vertices of vertex ',i,': ', str([vertex.index for vertex in onering]))




onering vertices of vertex  0 :  [1, 3, 2]


In [ ]:
icosphere = HalfedgeMesh('sphere4.off') #Use the library to generate a halfedge mesh from a .off file.
i=100
onering = find_onering(icosphere.vertices[i])#Does the onering have the expected size? (look at the mesh in Meshlab).
print ('onering vertices of vertex ',i,': ', str([vertex.index for vertex in onering]))

FileNotFoundError: [Errno 2] No such file or directory: 'sphere4.off'

## Onering Test: Mesh Editing

If your onering function is correct, you can use it to make edits to meshes.
 - What do you expect this code to do? Open the mesh in meshlab to check.
 - If you have time, try adapting it to make different patterns. For example, you could try editing all of the vertices in the two-ring.

In [ ]:
n = 42 #Magic number. Other magic numbers that make nice patterns are n=102, n=162 or n=642.

for i in range(n):
    origin_vertex = icosphere.vertices[i]
    onering = find_onering( origin_vertex )
    for vertex in onering:
        vertex.x = 1.02* vertex.x 
        vertex.y = 1.02* vertex.y 
        vertex.z = 1.02* vertex.z
    origin_vertex.x = 1.1* origin_vertex.x 
    origin_vertex.y = 1.1* origin_vertex.y 
    origin_vertex.z = 1.1* origin_vertex.z

icosphere.write_obj('edited_icosphere.obj')

## Extra Stuff

### Algorithm to Generate Halfedge Mesh from Vertices and Faces

If you finish everything else, learn how the library can automatically converts .off files into halfedge meshes.

This is the pseudocode for how we efficiently generate a halfedge mesh from a .off file. Link:
        http://stackoverflow.com/questions/15365471/initializing-half-edge-
        data-structure-from-vertices
The code assumes that the order of vertices in every face is consistent (e.g. always clockwise). See if you can work out which part would break if this assumption isn't true.

        map< pair<unsigned int, unsigned int>, HalfEdge* > Edges;

        for each face F
        {
            for each edge (u,v) of F
            {
                Edges[ pair(u,v) ] = new HalfEdge();
                Edges[ pair(u,v) ]->face = F;
            }
            for each edge (u,v) of F
            {
                set Edges[ pair(u,v) ]->nextHalfEdge to next half-edge in F
                if ( Edges.find( pair(v,u) ) != Edges.end() )
                {
                    Edges[ pair(u,v) ]->oppositeHalfEdge = Edges[ pair(v,u) ];
                    Edges[ pair(v,u) ]->oppositeHalfEdge = Edges[ pair(u,v) ];
            }
        }


24

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


22